In [2]:
import os
import sys
sys.path.append(os.path.abspath('') + '/..')

In [3]:
import pandas as pd 
import re
import json

from data_utils import getDF
from dataset_evaluation_utils import * 

from datetime import datetime, timezone

import seaborn as sns
import matplotlib.pyplot as plt
sns.set_style('whitegrid')

# import plotly.offline as py
# pd.options.plotting.backend = "plotly"
# py.init_notebook_mode() # graphs charts inline (IPython).

a4_dims = (11.7, 8.27)

# paths

In [10]:
dataset_folderpath = '../datasets/lastfm-1b/merged/'

dataset_name = 'LastFM1b'
dump_foldername ='lastfm1b_dump/'

# rule: what/which_data_set/sample_version/what/
images_path, output_path, heatmaps_path, diversity_graphpath, diversity_filepath = get_folderpaths(dump_foldername)

# dataset source

https://drive.google.com/drive/folders/1aXK8JSvOQMNjAplcp8TB2SBf-1uTn287

# tracks

In [16]:
ex_inter = pd.read_csv(dataset_folderpath+'lfm1b-tracks-example/lfm1b-tracks.inter', delimiter='\t')#, header=None)
ex_inter.head()

,user_id:token,tracks_id:token,timestamp:float,num_repeat:float
0,31435741,4,1385212958,13
1,31435741,17,1381867823,23
2,31435741,19,1381867431,22
3,31435741,53,1370977938,54
4,31435741,86,1369853851,84


tracks_id: the id of the tracks, and its type is token.\
name: the name of each track, and its type is token_seq.\
artists_id: the id of the artists to which the tracks belong, and its type is token.\

In [15]:
ex_item = pd.read_csv(dataset_folderpath+'lfm1b-tracks-example/lfm1b-tracks.item', delimiter='\t')#, header=None)
ex_item.head()

,tracks_id:token,name:token_seq,artists_id:token
0,1,A Matter of Time,3
1,2,Hangar 18,1
2,3,Up the Downstair,4
3,4,Mr. Carter (Featuring Jay-Z),2
4,5,Mixtaped,5


user_id: the id of the users, and its type is token.\
country: the country of the users, and its type is token.\
age: the age of the users, and its type is float.\
gender: the gender of the users, and its type is token.\
playcount: the number of playcount, and its type is float.\
registered_timestamp: the UNIX timestamp of users registering, and its type is float.\
novelty_artist_avg_month: the percentage of new artists listened to, averaged over time windows of 1 month, and its type is float.\
novelty_artist_avg_6months: the percentage of new artists listened to, averaged over time windows of 6 months, and its type is float.\
novelty_artist_avg_year: the percentage of new artists listened to, averaged over time windows of 1 year, and its type is float.\
mainstreaminess_avg_month: overlap between the user's listening history and an aggregate listening history of all users, averaged over time windows of 1 month, and its type is float.\
mainstreaminess_avg_6months: overlap between the user's listening history and an aggregate listening history of all users, averaged over time windows of 6 months, and its type is float.\
mainstreaminess_avg_year: overlap between the user's listening history and an aggregate listening history of all users, averaged over time windows of 1 year, and its type is float.\
mainstreaminess_global: overlap between the user's listening history and an aggregate listening history of all users, averaged over the entire period of the user's activity on Last.fm, and its type is float.\
cnt_listeningevents: total number of the user's listening events (playcounts) included in the dataset, and its type is float.\
cnt_distinct_tracks: number of unique tracks listened to by the user, and its type is float.\
cnt_distinct_artists: number of unique artists listened to by the user, and its type is float.\
cnt_listeningevents_per_week: average number of listening events per week, and its type is float.\
relative_le_per_weekday[1-7]: fraction of listening events for each weekday (starting on Monday) among all weekly plays, averaged over the user's entire listening history, and its type is float.\
relative le per hour[0-23]: fraction of listening events for each hour of the day (starting with the time span 0:00-0:59) among all 24 hours, averaged over the user's entire listening history, and its type is float.\

In [18]:
ex_user = pd.read_csv(dataset_folderpath+'lfm1b-tracks-example/lfm1b-tracks.user', delimiter='\t')#, header=None)
ex_user.head()

,user_id:token,country:token,age:float,gender:token,playcount:float,registered_timestamp:float,novelty_artist_avg_month:float,novelty_artist_avg_6months:float,novelty_artist_avg_year:float,mainstreaminess_avg_month:float,...,relative le per hour14:float,relative le per hour15:float,relative le per hour16:float,relative le per hour17:float,relative le per hour18:float,relative le per hour19:float,relative le per hour20:float,relative le per hour21:float,relative le per hour22:float,relative le per hour23:float
0,384,UK,35,m,42139,1035849600,0.276629,0.044439,0.309429,0.024655,...,0.0279,0.0410,0.0811,0.1506,0.0377,0.0298,0.0132,0.0007,0.0001,0.0151
1,1206,NaN,-1,n,33103,1035849600,0.437224,0.109671,0.513787,0.181991,...,0.0016,0.0115,0.0276,0.0662,0.0790,0.0838,0.0995,0.1195,0.0968,0.0839
2,2622,NaN,-1,NaN,2030,1037404800,0.604828,0.043923,0.698983,0.052310,...,0.1320,0.0493,0.0317,0.0470,0.0658,0.1127,0.1112,0.0680,0.0437,0.0254
3,2732,NaN,-1,n,147,1037577600,0.756973,0.020071,0.882801,0.005092,...,0.0598,0.0726,0.0171,0.0342,0.0769,0.1453,0.0470,0.0513,0.0085,0.0128
4,3653,UK,31,m,18504,1041033600,0.380005,0.045207,0.424411,0.042821,...,0.0541,0.0518,0.0564,0.0554,0.0849,0.0954,0.0833,0.0657,0.0471,0.0595


In [17]:
tracks_inter = pd.read_csv(dataset_folderpath+'lfm1b-tracks/lfm1b-tracks.inter', delimiter='\t')#, iterator=True)
tracks_inter.head()

# tracks_inter = pd.read_csv(dataset_folderpath+'lfm1b-tracks/lfm1b-tracks.inter', delimiter='\t', iterator=True)#, header=None)
# tracks_inter.get_chunk(100)

KeyboardInterrupt: 